In [45]:
import os, glob
import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import myutil.myutil as util

from evo.core import metrics
import evo.core.sync as sync
import evo.core.trajectory as evotraj

# Time to start considering the estimate
test_start = 4.9

wxx = 'w' +'95'
seq = f'cloud_avia_mid_{wxx}_dynxtrz_e5'

gtr_path = f'/media/tmn/mySataSSD1/Experiments/gptr_v2/sequences/'

# The path to the experiment log
xtrz_log = '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/sim_cloud_avia_mid_dynamic_extrinsics_gptr_two_lidar/run_00/time_0030/extrinsics_1.csv'

# The path to the gptr trajectory estimates
gptr_est_path = f'/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/gptr/'
print(gptr_est_path)

traj_trajlo_log_dir = f'/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/trajlo/'
print(traj_trajlo_log_dir)

traj_i2ekf_log_dir = f'/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/i2ekf/'
print(traj_i2ekf_log_dir)

traj_ctemlo_log_dir = f'/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/'
print(traj_ctemlo_log_dir)


# Lidar extrinsic
pose_B_L = [np.array([0,  45, 0,  0,   0,  0]),
            np.array([180, 0, 0, -0.5, 0, -0.25])]
T_B_L = []
for pose in pose_B_L:
	T = np.identity(4)
	T[0:3, 0:3] = util.eul2rotm(pose[0]/180*np.pi, pose[1]/180*np.pi, pose[2]/180*np.pi)
	T[0:3, 3] = pose[3:].T
	T_B_L.append(T)
	print("quat_B_L", util.rotm2quat(T[0:3, 0:3]))

T_L0_L1 = util.tfmult(util.tfinv(T_B_L[0]), T_B_L[1])
Rot01 = T_L0_L1[0:3, 0:3]
Pos01 = T_L0_L1[0:3, 3]
Eul01 = np.array(util.rotm2eul(Rot01))/math.pi*180
Qua01 = np.array(util.rotm2quat(Rot01))
print("xyz_L0_L1",  Pos01)
print("quat_L0_L1", Qua01)
print("ypr_L0_L1",  Eul01)

T_L1_L0 = util.tfinv(T_L0_L1)
Q_L1_L0 = np.array(util.rotm2quat(T_L1_L0[0:3, 0:3]))
print(f'extrinsic_T: [0, 0, 0, {T_L1_L0[0, 3]}, {T_L1_L0[1, 3]}, {T_L1_L0[2, 3]}]')
print(f'extrinsic_T: [0, 0, 0, {T_L1_L0[0, 3]}, {T_L1_L0[1, 3]}, {T_L1_L0[2, 3]}]')

# Lidar extrinsic slip
pose_B_L_slip = [np.array([0,  45, 0,  0,   0,  0]),
                 np.array([180, 0, 0, -0.5, 0, -0.35])]
T_B_L_slip = []
for pose in pose_B_L_slip:
	T_slip = np.identity(4)
	T_slip[0:3, 0:3] = util.eul2rotm(pose[0]/180*np.pi, pose[1]/180*np.pi, pose[2]/180*np.pi)
	T_slip[0:3, 3] = pose[3:].T
	T_B_L_slip.append(T_slip)

T_L0_L1_slip = util.tfmult(util.tfinv(T_B_L_slip[0]), T_B_L_slip[1])
Rot01_slip = T_L0_L1_slip[0:3, 0:3]
Pos01_slip = T_L0_L1_slip[0:3, 3]
Eul01_slip = np.array(util.rotm2eul(Rot01_slip))/math.pi*180
Qua01_slip = np.array(util.rotm2quat(Rot01_slip))
print("xyz slip", Pos01_slip)
print("ypr slip", Eul01_slip)


pose_W_L = [np.array([ 43,  48, 0,  0,    0,   0.70]),
            np.array([-134, 0,  0, -0.3, -0.3, 0.55])]
T_W_L = []
for lidx, pose in enumerate(pose_W_L):
	T = np.identity(4)
	T[0:3, 0:3] = util.eul2rotm(pose[0]/180*np.pi, pose[1]/180*np.pi, pose[2]/180*np.pi)
	T[0:3, 3] = pose[3:].T
	Q_W_L_init = util.eul2quat(pose[0]/180*np.pi, pose[1]/180*np.pi, pose[2]/180*np.pi)
	print(f"Q_W_L{lidx}_init", Q_W_L_init[[1, 2, 3, 0]])
	T_W_L.append(T)
 
T_L0_L1_init = util.tfmult(util.tfinv(T_W_L[0]), T_W_L[1])
Rot01_init = T_L0_L1_init[0:3, 0:3]
Pos01_init = T_L0_L1_init[0:3, 3]
Eul01_init = np.array(util.rotm2eul(Rot01_init))/math.pi*180
Qua01_init = np.array(util.rotm2quat(Rot01_init))
print("xyz_init", Pos01_init)
print("ypr_init", Eul01_init)

print("xyz_init_error", Pos01_init - Pos01)
print("ypr_init_error", Eul01_init - Eul01)

/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/gptr/
/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/trajlo/
/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/i2ekf/
/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/
quat_B_L [ 0.92387953 -0.          0.38268343 -0.        ]
quat_B_L [6.123234e-17 0.000000e+00 0.000000e+00 1.000000e+00]
xyz_L0_L1 [-0.1767767   0.         -0.53033009]
quat_L0_L1 [ 5.65713056e-17 -3.82683432e-01  0.00000000e+00  9.23879533e-01]
ypr_L0_L1 [ 1.8000000e+02  4.5000000e+01 -7.0167093e-15]
extrinsic_T: [0, 0, 0, -0.5, -6.123233995736766e-17, 0.24999999999999997]
extrinsic_T: [0, 0, 0, -0.5, -6.123233995736766e-17, 0.24999999999999997]
xyz slip [-0.10606602  0.         -0.60104076]
ypr slip [ 1.8000000e+02  4.5000000e+01 -7.0167093e-15]
Q_W_L0_init [-0.14906948  0.37843492  0.33481553  0.84997874]
Q_W_L1_init [ 0.          0.         -0.92050485  0.39073113]
xyz_init [-0.17224341 -0.0148066  -0.41546717]
ypr_init 

In [46]:
# Load the gtr logs

gtr_files = glob.glob(gtr_path + '/cloud_*/gtr/**/*.pcd', recursive=True)
gtr_files = sorted(gtr_files)

# Load the ground truth
df_gtr = pd.DataFrame(gtr_files, columns=['gtr_file'])

# Determine the sequence
def findGtrSeq(x):
    parts = x.split('/')
    for part in parts:
        if 'cloud_avia_mid_' in part:
            return part
    return 'unknown'

# Determine the lidar index
def findGtrLidx(x):
    if 'lidar_0' in x:
        return 0
    elif 'lidar_1' in x:
        return 1
    else:
        return -1

# Load the data
df_gtr['data'] = df_gtr['gtr_file'].apply(lambda x: np.loadtxt(x, delimiter=' ', skiprows=11))

# Identify the sequence name and lidar idx
df_gtr['seq']  = df_gtr['gtr_file'].apply(lambda x: findGtrSeq(x))
df_gtr['lidx'] = df_gtr['gtr_file'].apply(lambda x: findGtrLidx(x))
# df_gtr = df_gtr.groupby(['seqname', 'lidx'])

# print(df_gtr)

# Calculate the ATE
def calculate_metric(traj_gtr, traj_est):
    metric = metrics.APE(pose_relation=metrics.PoseRelation.translation_part)
    metric.process_data((traj_gtr, traj_est))
    return float(metric.get_result(ref_name='reference', est_name='estimate').stats['rmse'])

In [47]:

# Access gptr method

# Determine the sequence
def findSeq(x):
    parts = x.split('/')
    for part in parts:
        if 'cloud_avia_mid_' in part:
            # print(part)
            return part.replace('_coupled', '')
    return 'unknown'

def findWxx(x):
    parts = x.split('_')
    for part in parts:
        if 'w' in part:
            return part
    return 'unknown'

def findMethod(x):
    parts = x.split('/')
    for part in parts:
        if 'exp_' in part:
            method = part
            method = method.replace('exp_', '')
            # print(method)
            if '0' in method:
                method = method.replace('0', 'CF')
            elif '1' in method:
                method = method.replace('1', 'AP')
            return method
    return 'unknown'

def findRun(x):
    parts = x.split('/')
    for part in parts:
        if 'run_' in part:
            run = int(part.replace('run_', ''))
            return run
    return -1

def findTime(x):
    parts = x.split('/')
    for part in parts:
        if 'time_' in part:
            time = float(part.replace('time_', ''))
            return time
    return 0

def findLidx(x):
    parts = x.split('/')
    for part in parts:
        if('gptraj_' in part):
            return int(part.replace('gptraj_', '').replace('.csv', ''))

def findRMSE(x):
    
    logfile = x['logfile']
    
    # rmse = -1
    # with open(logfile) as file:
    #     header = file.readline().strip()
    #     # print(header)
    #     parts = header.split(';')
    #     for part in parts:
    #         if "RMSE:" in part:
    #             rmse = float(part.replace('RMSE:', ''))*1000
    #             # print(rmse)
    #             return rmse
    # return rmse

    lidx = x['lidx']
    seq  = x['seq']

    # Load the trajectory
    traj_est_data = np.loadtxt(logfile, delimiter=',', skiprows=1)
    traj_gtr_data = df_gtr[ (df_gtr['lidx'] == int(lidx)) & (df_gtr['seq'] == seq) ]['data'].iloc[0]
    
    # Get the traj_est
    tidx = list(np.where(traj_est_data[:, 1] > test_start)[0])
    traj_est = evotraj.PoseTrajectory3D(positions_xyz=traj_est_data[tidx, :][:, [12, 13, 14]], orientations_quat_wxyz=traj_est_data[tidx, :][:, [5, 2, 3, 4]], timestamps=traj_est_data[tidx, 1])
    t_est_max = traj_est.timestamps.max()

    # Get the traj gtr
    traj_gtr = traj_gtr_data
    traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr[:, 0:3], orientations_quat_wxyz=traj_gtr[:, [8, 5, 6, 7]], timestamps=traj_gtr[:, 3])
    t_gtr_max = traj_gtr.timestamps.max()
    
    # if abs(t_est_max - t_gtr_max) > 1.0:
    #     # print('divergence', t_est_max, t_gtr_max, x)
    #     return -1

    traj_est, traj_gtr = sync.associate_trajectories(traj_est, traj_gtr, max_diff=0.2)
    traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_est.positions_xyz - traj_gtr.positions_xyz, \
                                        orientations_quat_wxyz=traj_est.orientations_quat_wxyz, timestamps=traj_est.timestamps)

    # Find the RMSE
    return calculate_metric(traj_est, traj_gtr)

def findXtrzLog(x):
    dirname = os.path.dirname(x)
    extrinsic_log = glob.glob(dirname + '/**/extrinsics_*.csv', recursive=True)[0]
    return extrinsic_log

traj_gptrc_logs = glob.glob(gptr_est_path + '/**/gptraj_*.csv', recursive=True)
traj_gptrc_logs = [log for log in traj_gptrc_logs if 'odom' not in log]

# print(traj_gptrc_logs)

df_gptr = pd.DataFrame(traj_gptrc_logs, columns=['logfile'])
df_gptr['seq']    = df_gptr['logfile'].apply(lambda x : findSeq(x))
df_gptr['wxx']    = df_gptr['seq'].apply(lambda x : findWxx(x))
df_gptr['method'] = df_gptr['logfile'].apply(lambda x : findMethod(x))
df_gptr['run']    = df_gptr['logfile'].apply(lambda x : findRun(x))
df_gptr = df_gptr[df_gptr['run'] == 1]
df_gptr['lidx']   = df_gptr['logfile'].apply(lambda x : findLidx(x))
df_gptr['time']   = df_gptr['logfile'].apply(lambda x : findTime(x))
# df_gptr = df_gptr.drop(['seq'], axis=1)
df_gptr = df_gptr.loc[df_gptr.groupby(['wxx', 'method', 'run', 'lidx'])['time'].idxmax()]
df_gptr['rmse'] = df_gptr.apply(lambda x : findRMSE(x), axis=1)

df_gptr_pvt = df_gptr.pivot(index=['wxx', 'lidx'], columns=['method'], values='rmse')
print(df_gptr_pvt)


method      SE3_AP    SE3_CF  SO3xR3_AP  SO3xR3_CF
wxx lidx                                          
w25 0     0.034330  0.034353   0.034288   0.034252
    1     0.039151  0.039157   0.039224   0.039193
w35 0     0.034357  0.034333   0.034076   0.034104
    1     0.040469  0.040483   0.040523   0.040537
w45 0     0.033604  0.033655   0.033227   0.033192
    1     0.041064  0.041038   0.041230   0.041248
w55 0     0.035229  0.035207   0.034688   0.034667
    1     0.042730  0.042701   0.043137   0.043184
w65 0     0.034843  0.034785   0.034185   0.034231
    1     0.043613  0.043616   0.044382   0.044328
w75 0     0.035077  0.035068   0.034031   0.033961
    1     0.044321  0.044348   0.045433   0.045460
w85 0     0.037199  0.037346   0.035947   0.035947
    1     0.045747  0.045708   0.046794   0.046845
w95 0     0.038389  0.038503   0.037383   0.037385
    1     0.047423  0.047475   0.048964   0.048913


In [48]:
# Access TrajLO

def findRMSE(x):
    
    seq  = x['seq']
    lidx = x['lidx']
    
    traj_trajlo_data = x['data']
    traj_gtr_data = df_gtr[ (df_gtr['lidx'] == lidx) & (df_gtr['seq'] == seq) ]['data'].iloc[0]
    
    tidx = list(np.where( (traj_trajlo_data[:, 0] > test_start) )[0])
    traj_trajlo = evotraj.PoseTrajectory3D(positions_xyz=traj_trajlo_data[tidx, :][:, [1, 2, 3]], orientations_quat_wxyz=traj_trajlo_data[tidx, :][:, [7, 4, 5, 6]], timestamps=traj_trajlo_data[tidx, 0])
    traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr_data[:, 0:3], orientations_quat_wxyz=traj_gtr_data[:, [8, 5, 6, 7]], timestamps=traj_gtr_data[:, 3])
    traj_trajlo, traj_gtr = sync.associate_trajectories(traj_trajlo, traj_gtr, max_diff=0.2)
    traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_trajlo.positions_xyz - traj_gtr.positions_xyz, \
                                        orientations_quat_wxyz=traj_trajlo.orientations_quat_wxyz, timestamps=traj_trajlo.timestamps)
    rmse = calculate_metric(traj_trajlo, traj_gtr)
    return rmse, traj_err

# Load the trajectory
traj_trajlo_logs = glob.glob(traj_trajlo_log_dir + '/**/*.txt', recursive=True)
df_trajlo = pd.DataFrame(traj_trajlo_logs, columns=['logfile'])
df_trajlo['seq']    = df_trajlo['logfile'].apply(lambda x : findSeq(x))
df_trajlo['wxx']    = df_trajlo['seq'].apply(lambda x : findWxx(x))
df_trajlo['method'] = df_trajlo['logfile'].apply(lambda x : 'TRAJLO')
df_trajlo['lidx']   = df_trajlo['logfile'].apply(lambda x : 0 if 'lidar0' in x else 1)
df_trajlo['data']   = df_trajlo['logfile'].apply(lambda x : np.loadtxt(x, delimiter=' ', skiprows=1))
df_trajlo[['rmse', 'error']] = df_trajlo.apply(lambda x : findRMSE(x), axis=1, result_type='expand')

# Extract the exp with minimum rmse for each lidx
df_trajlo_pvt = df_trajlo.pivot(index=['wxx', 'lidx'], columns='method', values='rmse')
print(df_trajlo_pvt)

    

method      TRAJLO
wxx lidx          
w25 0     0.063548
    1     0.058503
w35 0     0.063281
    1     0.058568
w45 0     0.063206
    1     0.059299
w55 0     0.062613
    1     0.062421
w65 0     0.062999
    1     0.065711
w75 0     0.062319
    1     0.071514
w85 0     0.062892
    1     0.080037
w95 0     0.064089
    1     0.090541


In [49]:
# Access the i2ekf traj

def findRMSE(x):
    
    traj_i2ekf_data = x['data']
    
    lidx = x['lidx']
    seq = x['seq']
    
    traj_gtr_data = df_gtr[ (df_gtr['lidx'] == lidx) & (df_gtr['seq'] == seq) ]['data'].iloc[0]
    
    tidx = list(np.where( (traj_i2ekf_data[:, 0] > test_start) )[0])
    traj_i2ekf = evotraj.PoseTrajectory3D(positions_xyz=traj_i2ekf_data[tidx, :][:, [1, 2, 3]], orientations_quat_wxyz=traj_i2ekf_data[tidx, :][:, [7, 4, 5, 6]], timestamps=traj_i2ekf_data[tidx, 0])
    traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr_data[:, 0:3], orientations_quat_wxyz=traj_gtr_data[:, [8, 5, 6, 7]], timestamps=traj_gtr_data[:, 3])
    traj_i2ekf, traj_gtr = sync.associate_trajectories(traj_i2ekf, traj_gtr, max_diff=0.2)
    traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_i2ekf.positions_xyz - traj_gtr.positions_xyz, \
                                        orientations_quat_wxyz=traj_i2ekf.orientations_quat_wxyz, timestamps=traj_i2ekf.timestamps)
    rmse = calculate_metric(traj_i2ekf, traj_gtr)
    return rmse, traj_err

# Load the trajectory
traj_i2ekf_logs = glob.glob(traj_i2ekf_log_dir + '/**/i2ekf_lidar*.csv', recursive=True)
df_i2ekf = pd.DataFrame(traj_i2ekf_logs, columns=['logfile'])
df_i2ekf['seq']    = df_i2ekf['logfile'].apply(lambda x : findSeq(x))
df_i2ekf['wxx']    = df_i2ekf['seq'].apply(lambda x : findWxx(x))
df_i2ekf['method'] = df_i2ekf['logfile'].apply(lambda x : 'I2EKF')
df_i2ekf['lidx']   = df_i2ekf['logfile'].apply(lambda x : 0 if 'lidar0' in x else 1)
df_i2ekf['data']   = df_i2ekf['logfile'].apply(lambda x : np.loadtxt(x, skiprows=1))
df_i2ekf[['rmse', 'error']] = df_i2ekf.apply(lambda x : findRMSE(x), axis=1, result_type='expand')

# lidars = list(set(df_i2ekf['lidx'].tolist()))
# # Calculate the variance of trajlo est
# import statistics
# for lidx in lidars:
#     print(f"variance {lidx}", statistics.variance(df_i2ekf[ df_i2ekf['lidx'] == lidx]['rmse'].tolist() ))
    
# Extract the exp with minimum rmse for each lidx
df_i2ekf_pvt = df_i2ekf.pivot(index=['wxx', 'lidx'], columns='method', values='rmse')
print(df_i2ekf_pvt)

method       I2EKF
wxx lidx          
w25 0     0.057117
    1     0.049547
w35 0     0.057563
    1     0.050581
w45 0     0.057951
    1     0.053148
w55 0     0.058306
    1     0.056407
w65 0     0.059492
    1     0.062078
w75 0     0.061711
    1     0.069271
w85 0     0.063214
    1     0.075599
w95 0     0.067106
    1     0.085625


In [50]:
# Access the ctemlo traj

def findRMSE(x):
    
    traj_ctemlo_data = x['data']
    
    lidx = x['lidx']
    seq = x['seq']
    
    traj_gtr_data = df_gtr[ (df_gtr['lidx'] == lidx) & (df_gtr['seq'] == seq) ]['data'].iloc[0]
    
    tidx = list(np.where( (traj_ctemlo_data[:, 0] > test_start) )[0])
    traj_ctemlo = evotraj.PoseTrajectory3D(positions_xyz=traj_ctemlo_data[tidx, :][:, [1, 2, 3]], orientations_quat_wxyz=traj_ctemlo_data[tidx, :][:, [7, 4, 5, 6]], timestamps=traj_ctemlo_data[tidx, 0])
    traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr_data[:, 0:3], orientations_quat_wxyz=traj_gtr_data[:, [8, 5, 6, 7]], timestamps=traj_gtr_data[:, 3])
    traj_ctemlo, traj_gtr = sync.associate_trajectories(traj_ctemlo, traj_gtr, max_diff=0.2)
    traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_ctemlo.positions_xyz - traj_gtr.positions_xyz, \
                                        orientations_quat_wxyz=traj_ctemlo.orientations_quat_wxyz, timestamps=traj_ctemlo.timestamps)
    rmse = calculate_metric(traj_ctemlo, traj_gtr)
    return rmse, traj_err

# Load the trajectory
traj_ctemlo_logs = glob.glob(traj_ctemlo_log_dir + '/**/ctemlo_lidar*.csv', recursive=True)

print(traj_ctemlo_logs)

df_ctemlo = pd.DataFrame(traj_ctemlo_logs, columns=['logfile'])
df_ctemlo['seq']    = df_ctemlo['logfile'].apply(lambda x : findSeq(x))
df_ctemlo['wxx']    = df_ctemlo['seq'].apply(lambda x : findWxx(x))
df_ctemlo['method'] = df_ctemlo['logfile'].apply(lambda x : 'CTEMLO')
df_ctemlo['lidx']   = df_ctemlo['logfile'].apply(lambda x : 0 if 'lidar0' in x else 1)
df_ctemlo['data']   = df_ctemlo['logfile'].apply(lambda x : np.loadtxt(x, skiprows=1))
df_ctemlo[['rmse', 'error']] = df_ctemlo.apply(lambda x : findRMSE(x), axis=1, result_type='expand')

# lidars = list(set(df_i2ekf['lidx'].tolist()))
# # Calculate the variance of trajlo est
# import statistics
# for lidx in lidars:
#     print(f"variance {lidx}", statistics.variance(df_i2ekf[ df_i2ekf['lidx'] == lidx]['rmse'].tolist() ))
    
# Extract the exp with minimum rmse for each lidx
df_ctemlo_pvt = df_ctemlo.pivot(index=['wxx', 'lidx'], columns='method', values='rmse')
print(df_ctemlo_pvt)

['/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w25_dynxtrz_e5/ctemlo_lidar0.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w25_dynxtrz_e5/ctemlo_lidar1.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w35_dynxtrz_e5/ctemlo_lidar0.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w35_dynxtrz_e5/ctemlo_lidar1.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w45_dynxtrz_e5/ctemlo_lidar0.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w45_dynxtrz_e5/ctemlo_lidar1.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w55_dynxtrz_e5/ctemlo_lidar0.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/ctemlo/cloud_avia_mid_w55_dynxtrz_e5/ctemlo_lidar1.csv', '/media/tmn/mySataSSD1/Experiments/gptr_v2/logs/lio/sim_exp/cte

In [56]:
# Combine the results

df_trajlo = df_trajlo[['wxx', 'lidx', 'method', 'rmse']]
df_i2ekf  = df_i2ekf[['wxx', 'lidx', 'method', 'rmse']]
df_ctemlo = df_ctemlo[['wxx', 'lidx', 'method', 'rmse']]
df_gptr   = df_gptr[['wxx', 'lidx', 'method', 'rmse']]

df_combibed = pd.concat([df_trajlo, df_i2ekf, df_ctemlo, df_gptr], axis=0)
df_combibed = df_combibed.sort_values(by='lidx')
df_combibed = df_combibed.pivot(index=['wxx', 'lidx'], columns='method', values='rmse')

cols = ['I2EKF', 'TRAJLO', 'CTEMLO'] + [col for col in df_combibed.columns if col not in ['TRAJLO', 'I2EKF'] and 'CF' in col ]
df_combibed = df_combibed[cols]
# df_combined = df_combibed['CF' in df_combibed['method']]

df_combibed = df_combibed.round(4)
df_combibed[df_combibed > 1.0] = np.inf

print(df_combibed)

def makeLatexTable(df_):
    df = df_
    for idx, row in df.iterrows():
        sorted_cols = row.sort_values().index  # columns sorted by value
        min_col = sorted_cols[0]
        second_min_col = sorted_cols[1]
        for col in df.columns:
            val = row[col]
            if col == min_col:
                df.at[idx, col] = f"\\textbf{{{val}}}"
            elif col == second_min_col:
                df.at[idx, col] = f"\\underline{{{val}}}"
            else:
                df.at[idx, col] = f"{val}"

    # Export to LaTeX
    latex_str = df.to_latex(escape=False)
    return latex_str

latextable = makeLatexTable(df_combibed)\
            .replace("textbf", "bf")\
            .replace("underline", "ul")\
            .replace('multirow', 'mr')\
            .replace('multicol', 'mc')\
            .replace('\mr[t]', '\mr[c]')\
            .replace('w', '0.')\
            .replace('inf', 'x')

print(latextable)

method     I2EKF  TRAJLO  CTEMLO  SE3_CF  SO3xR3_CF
wxx lidx                                           
w25 0     0.0571  0.0635  0.0599  0.0344     0.0343
    1     0.0495  0.0585  0.0619  0.0392     0.0392
w35 0     0.0576  0.0633     inf  0.0343     0.0341
    1     0.0506  0.0586     inf  0.0405     0.0405
w45 0     0.0580  0.0632     inf  0.0337     0.0332
    1     0.0531  0.0593     inf  0.0410     0.0412
w55 0     0.0583  0.0626     inf  0.0352     0.0347
    1     0.0564  0.0624     inf  0.0427     0.0432
w65 0     0.0595  0.0630     inf  0.0348     0.0342
    1     0.0621  0.0657     inf  0.0436     0.0443
w75 0     0.0617  0.0623  0.0574  0.0351     0.0340
    1     0.0693  0.0715  0.0658  0.0443     0.0455
w85 0     0.0632  0.0629  0.0623  0.0373     0.0359
    1     0.0756  0.0800  0.0712  0.0457     0.0468
w95 0     0.0671  0.0641  0.0654  0.0385     0.0374
    1     0.0856  0.0905  0.0749  0.0475     0.0489
\begin{tabular}{lllllll}
\toprule
 & method & I2EKF & TRAJLO & C

/tmp/ipykernel_2944678/502141106.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0571' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, col] = f"{val}"
/tmp/ipykernel_2944678/502141106.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0635' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, col] = f"{val}"
/tmp/ipykernel_2944678/502141106.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0599' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, col] = f"{val}"
/tmp/ipykernel_2944678/502141106.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will

In [52]:
# # Load the extrinsic data
# xtrz_data = np.loadtxt(xtrz_log, delimiter=',', skiprows=1)
# txtrz = xtrz_data[:, 0]

# # Calculate the position data
# p_L0_L1_est = xtrz_data[:, 1:4]
# p_L0_L1_err = np.zeros(p_L0_L1_est.shape)
# p_L0_L1_gtr = np.zeros(p_L0_L1_est.shape)
# for idx, t in enumerate(txtrz):
#     if 10 < t < 20:
#         p_L0_L1_gtr[idx, :] = p_L0_L1_est[idx, :] - Pos01_slip
#         p_L0_L1_gtr[idx, :] = Pos01_slip
#     else:
#         p_L0_L1_err[idx, :] = p_L0_L1_est[idx, :] - Pos01
#         p_L0_L1_gtr[idx, :] = Pos01

# # Calculate the oritation error
# Q_L0_L1_est = []
# E_L0_L1_est = []
# E_L0_L1_err = []
# for idx, t in enumerate(txtrz):
#     q = xtrz_data[idx, [7, 4, 5, 6]]
#     Q_L0_L1_est.append(util.quat2rotm(q))
#     E_L0_L1_est.append(list(util.quat2eul(q)))
#     E_L0_L1_err.append(list(util.rotm2eul(Q_L0_L1_est[-1].transpose()@Rot01)))

# E_L0_L1_est = np.array(E_L0_L1_est).reshape((-1, 3))
# E_L0_L1_err = np.array(E_L0_L1_err).reshape((-1, 3))

# # Plot the error
# plt.rcParams.update({'font.size':18})
# fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 4))

# ax.plot(txtrz, p_L0_L1_est[:, 0], linewidth=2, color='r', label='x est.')
# ax.plot(txtrz, p_L0_L1_gtr[:, 0], linewidth=2, color='r', label='x GT', linestyle='--')

# ax.plot(txtrz, p_L0_L1_est[:, 1], linewidth=2, color='g', label='y est.')
# ax.plot(txtrz, p_L0_L1_gtr[:, 1], linewidth=2, color='g', label='y GT', linestyle='--')

# ax.plot(txtrz, p_L0_L1_est[:, 2], linewidth=2, color='b', label='z est.')
# ax.plot(txtrz, p_L0_L1_gtr[:, 2], linewidth=2, color='b', label='z GT', linestyle='--')

# ax.text(x=0.1, y=0.8, s=f'Final error: \n   ({p_L0_L1_err[-1, 0]:.3f}, {p_L0_L1_err[-1, 1]:.3f}, {p_L0_L1_err[-1, 2]:.3f})', transform=ax.transAxes)

# # ax.set_xlim([5, 15])
# ax.grid('on')
# ax.legend(ncol=3, loc='upper left', bbox_to_anchor=(0.05, 0.65))

# ax.set_xlabel('Time [s]')
# ax.set_ylabel('Trans. Extrinsics [m]')
# # plt.tight_layout()
# fig.canvas.draw()
# fig.savefig('trans_extrinsics_error.jpg', bbox_inches='tight', dpi=300)

# # Plot the error
# plt.rcParams.update({'font.size':18})
# fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 4))

# ax.plot(txtrz, E_L0_L1_err[:, 0]/np.pi*180, linewidth=2, color='r', label='yaw')
# ax.plot(txtrz, E_L0_L1_err[:, 1]/np.pi*180, linewidth=2, color='g', label='pitch')
# ax.plot(txtrz, E_L0_L1_err[:, 2]/np.pi*180, linewidth=2, color='b', label='raw')

# ax.grid('on')
# ax.legend(ncol=3, bbox_to_anchor=(0.4, 0.35))

# ax.set_xlabel('Time [s]')
# ax.set_ylabel('Rot. Extr. Est. Error [deg]')

# # plt.tight_layout()
# fig.canvas.draw()
# fig.savefig('rot_extrinsics_error.jpg', bbox_inches='tight', dpi=300)

In [53]:
import pandas as pd

# Sample DataFrame
data = {
    'TRAJLO': [0.064545, 0.057270],
    'I2EKF': [0.057025, 0.049441],
    'SE3_AP': [0.033506, 0.039525],
    'SE3_CF': [0.033593, 0.039484],
    'SO3xR3_AP': [0.033498, 0.039336],
    'SO3xR3_CF': [0.033639, 0.039300]
}
index = [0, 1]
df = pd.DataFrame(data, index=index)
df.index.name = 'lidx'

print("Original DataFrame:")
print(df)

# Find the minimum value per row
min_idx = df.idxmin(axis=1)

# Create a copy for LaTeX formatting
df_latex = df.copy()

# Apply bold formatting for the minimum value in each row
for idx in df.index:
    for col in df.columns:
        val = df.at[idx, col]
        if col == min_idx[idx]:
            df_latex.at[idx, col] = f"\\textbf{{{val:.6f}}}"
        else:
            df_latex.at[idx, col] = f"{val:.6f}"

# Export to LaTeX
latex_str = df_latex.to_latex(escape=False)

print("\nGenerated LaTeX Table:")
print(latex_str)


Original DataFrame:
        TRAJLO     I2EKF    SE3_AP    SE3_CF  SO3xR3_AP  SO3xR3_CF
lidx                                                              
0     0.064545  0.057025  0.033506  0.033593   0.033498   0.033639
1     0.057270  0.049441  0.039525  0.039484   0.039336   0.039300

Generated LaTeX Table:
\begin{tabular}{lllllll}
\toprule
 & TRAJLO & I2EKF & SE3_AP & SE3_CF & SO3xR3_AP & SO3xR3_CF \\
lidx &  &  &  &  &  &  \\
\midrule
0 & 0.064545 & 0.057025 & 0.033506 & 0.033593 & \textbf{0.033498} & 0.033639 \\
1 & 0.057270 & 0.049441 & 0.039525 & 0.039484 & 0.039336 & \textbf{0.039300} \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2944678/1313471214.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.064545' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_latex.at[idx, col] = f"{val:.6f}"
/tmp/ipykernel_2944678/1313471214.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.057025' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_latex.at[idx, col] = f"{val:.6f}"
/tmp/ipykernel_2944678/1313471214.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.033506' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_latex.at[idx, col] = f"{val:.6f}"
/tmp/ipykernel_2944678/1313471214.py:32: FutureWarning: Setting an item of i

In [54]:
# # Load the trajectory
# traj_est1lidar_data = []
# for log in traj_est1lidar_logs:
#     traj_est1lidar_data.append(np.loadtxt(log, delimiter=',', skiprows=1))

# traj_gtr_data = []
# for log in traj_gtr_logs:
#     traj_gtr_data.append(np.loadtxt(log, delimiter=' ', skiprows=11))

# for lidx in range(len(traj_est1lidar_data)):
    
#     # Get the traj_est
#     traj_est1lidar = traj_est1lidar_data[lidx]
#     tidx = list(np.where(traj_est1lidar[:, 1] > test_start)[0])
#     traj_est1lidar = evotraj.PoseTrajectory3D(positions_xyz=traj_est1lidar[tidx, :][:, [12, 13, 14]], orientations_quat_wxyz=traj_est1lidar[tidx, :][:, [5, 2, 3, 4]], timestamps=traj_est1lidar[tidx, 1])

#     # Get the traj gtr
#     traj_gtr = traj_gtr_data[lidx]
#     traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr[:, 0:3], orientations_quat_wxyz=traj_gtr[:, [8, 5, 6, 7]], timestamps=traj_gtr[:, 3])

#     traj_est1lidar, traj_gtr = sync.associate_trajectories(traj_est1lidar, traj_gtr, max_diff=0.2)
#     traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_est1lidar.positions_xyz - traj_gtr.positions_xyz, \
#                                         orientations_quat_wxyz=traj_est1lidar.orientations_quat_wxyz, timestamps=traj_est1lidar.timestamps)
    
#     # Find the RMSE
#     rmse = calculate_metric(traj_est1lidar, traj_gtr)
    
#     plt.rcParams.update({'font.size':18})
#     fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 4))
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 0], 'r', linewidth=2, label='x')
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 1], 'g', linewidth=2, label='y')
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 2], 'b', linewidth=2, label='z')

#     # Stamp the ATE
#     ax.text(0, -0.2, f'GPTR-U, Lidar {lidx}. APE: {rmse:.5f}', transform=ax.transAxes)

#     ax.set_xlim([np.min(traj_est1lidar.timestamps), np.max(traj_est1lidar.timestamps-1.0)])
#     # ax.set_ylim([-0.06, 0.06])
#     ax.legend(ncols=3)
#     ax.grid('on')
#     ax.set_xlabel('Time [s]')
#     ax.set_ylabel('Error [m]')


In [55]:
# # Load the trajectory
# traj_estkf_data = []
# for log in traj_estkf_logs:
#     traj_estkf_data.append(np.loadtxt(log, delimiter=' ', skiprows=11))

# traj_gtr_data = []
# for log in traj_gtr_logs:
#     traj_gtr_data.append(np.loadtxt(log, delimiter=' ', skiprows=11))

# for lidx in range(len(traj_estkf_data)):
    
#     # Get the traj_estkf
#     traj_estkf = traj_estkf_data[lidx]
#     tidx = list(np.where(traj_estkf[:, 4] > test_start)[0])
#     traj_estkf = evotraj.PoseTrajectory3D(positions_xyz=traj_estkf[tidx, :][:, [0, 1, 2]], orientations_quat_wxyz=traj_estkf[tidx, :][:, [8, 5, 6, 7]], timestamps=traj_estkf[tidx, 4])

#     # Get the traj gtr
#     traj_gtr = traj_gtr_data[lidx]
#     traj_gtr = evotraj.PoseTrajectory3D(positions_xyz=traj_gtr[:, 0:3], orientations_quat_wxyz=traj_gtr[:, [8, 5, 6, 7]], timestamps=traj_gtr[:, 3])

#     traj_estkf, traj_gtr = sync.associate_trajectories(traj_estkf, traj_gtr, max_diff=0.2)
#     traj_err = evotraj.PoseTrajectory3D(positions_xyz=traj_estkf.positions_xyz - traj_gtr.positions_xyz, \
#                                         orientations_quat_wxyz=traj_estkf.orientations_quat_wxyz, timestamps=traj_estkf.timestamps)
    
#     # Find the RMSE
#     rmse = calculate_metric(traj_estkf, traj_gtr)
    
#     plt.rcParams.update({'font.size':18})
#     fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 4))
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 0], 'r', linewidth=2, label='x')
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 1], 'g', linewidth=2, label='y')
#     ax.plot(traj_err.timestamps, traj_err.positions_xyz[:, 2], 'b', linewidth=2, label='z')

#     # Stamp the ATE
#     ax.text(0, -0.2, f'IEKF, Lidar {lidx}. APE: {rmse:.5f}', transform=ax.transAxes)

#     ax.set_xlim([np.min(traj_estkf.timestamps), np.max(traj_estkf.timestamps-1.0)])
#     # ax.set_ylim([-0.06, 0.06])
#     ax.legend(ncols=3)
#     ax.grid('on')
#     ax.set_xlabel('Time [s]')
#     ax.set_ylabel('Error [m]')